In [1]:
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from Pytorch_dataset import DimReduction_dataset
from Discharge_model import DimReduction_1
from early_stopping import EarlyStopping
from torch.utils.tensorboard import SummaryWriter

c:\Users\smcmlab4\Anaconda3\envs\python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 32

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('cuda is available')

# load dataset
eol_train_dataset = DimReduction_dataset(train=True, pred_target='EOL', norm=True)
eol_test_dataset = DimReduction_dataset(train=False, pred_target='EOL', norm=True)

eol_train_loader = DataLoader(eol_train_dataset, batch_size=batch_size, shuffle=True)
eol_test_loader = DataLoader(eol_test_dataset, batch_size=batch_size, shuffle=False)

cuda is available


In [3]:
def train_model(model, writer,
                num_epochs=500,
                lr=1e-3,
                weight_decay=1e-3,
                cosine_period=20,
                min_lr=1e-6,
                delta_huber=1,
                patience=20):

    optimizer = optim.AdamW(model.parameters(), lr=lr, amsgrad=True, weight_decay=weight_decay) # Adam with weight decay
    scheduler = CosineAnnealingLR(optimizer, T_max=cosine_period, eta_min=min_lr) # 依照cosine週期衰減
    criterion = nn.HuberLoss(delta=delta_huber) # combines advantages of both L1Loss and MSELoss

    start = time.time()

    train_losses = []
    valid_losses = []
    avg_train_losses = []
    avg_valid_losses = []
    n_total_steps = len(eol_train_loader)

    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True)

    for epoch in range(num_epochs):

        ##### Training loop #####
        model.train() # prep model for training
        for i, (inputs, targets) in enumerate(eol_train_loader):
            inputs = inputs.to(device)
            targets = targets.view(-1, 1).to(device)

            # forward
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            
        # update lr
        scheduler.step()

        ##### Validation loop #####
        model.eval() # prep model for evaluation
        for inputs, targets in eol_test_loader:
            inputs = inputs.to(device)
            targets = targets.view(-1, 1).to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            valid_losses.append(loss.item())
        
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        print(f'[Epoch {epoch+1}/{num_epochs}] train_loss: {train_loss:.2f}, valid_loss: {valid_loss:.2f}')

        # tensorboard
        writer.add_scalar('train loss', train_loss, epoch * n_total_steps +i) # global step
        writer.add_scalar('valid loss', valid_loss, epoch * n_total_steps +i)

        # clear lists to track next epoch
        train_losses = []
        valid_losses = []

        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('Checkpoints/checkpoint.pt'))

    end = time.time()
    print(f'Training is end. Total trainig time: {(end-start)/60:.1f} minutes')

    return  model, avg_train_losses, avg_valid_losses

# New

In [4]:
# TensorBoard
writer = SummaryWriter('runs/eol_new_1')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=50, lr=1e-2, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.3)

[Epoch 1/500] train_loss: 55.08, valid_loss: 51.82
Validation loss decreased (inf --> 51.818908).  Saving model ...
[Epoch 2/500] train_loss: 43.15, valid_loss: 50.11
Validation loss decreased (51.818908 --> 50.114452).  Saving model ...
[Epoch 3/500] train_loss: 42.21, valid_loss: 65.10
EarlyStopping counter: 1 out of 50
[Epoch 4/500] train_loss: 41.15, valid_loss: 50.52
EarlyStopping counter: 2 out of 50
[Epoch 5/500] train_loss: 39.63, valid_loss: 50.78
EarlyStopping counter: 3 out of 50
[Epoch 6/500] train_loss: 38.17, valid_loss: 54.84
EarlyStopping counter: 4 out of 50
[Epoch 7/500] train_loss: 35.45, valid_loss: 52.59
EarlyStopping counter: 5 out of 50
[Epoch 8/500] train_loss: 32.64, valid_loss: 50.77
EarlyStopping counter: 6 out of 50
[Epoch 9/500] train_loss: 29.68, valid_loss: 52.04
EarlyStopping counter: 7 out of 50
[Epoch 10/500] train_loss: 27.58, valid_loss: 54.95
EarlyStopping counter: 8 out of 50
[Epoch 11/500] train_loss: 26.74, valid_loss: 55.76
EarlyStopping counter

In [5]:
file = 'Model/eol_model_new_1.pth'
torch.save(model, file)
print(file)

Model/eol_model_new_1.pth


## norm=True

In [4]:
# TensorBoard
writer = SummaryWriter('runs/eol_new_2')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=50, lr=1e-2, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.3)

[Epoch 1/500] train_loss: 0.21, valid_loss: 0.15
Validation loss decreased (inf --> 0.145063).  Saving model ...
[Epoch 2/500] train_loss: 0.13, valid_loss: 0.11
Validation loss decreased (0.145063 --> 0.111447).  Saving model ...
[Epoch 3/500] train_loss: 0.11, valid_loss: 0.11
EarlyStopping counter: 1 out of 50
[Epoch 4/500] train_loss: 0.09, valid_loss: 0.13
EarlyStopping counter: 2 out of 50
[Epoch 5/500] train_loss: 0.09, valid_loss: 0.13
EarlyStopping counter: 3 out of 50
[Epoch 6/500] train_loss: 0.08, valid_loss: 0.14
EarlyStopping counter: 4 out of 50
[Epoch 7/500] train_loss: 0.08, valid_loss: 0.15
EarlyStopping counter: 5 out of 50
[Epoch 8/500] train_loss: 0.07, valid_loss: 0.13
EarlyStopping counter: 6 out of 50
[Epoch 9/500] train_loss: 0.07, valid_loss: 0.14
EarlyStopping counter: 7 out of 50
[Epoch 10/500] train_loss: 0.06, valid_loss: 0.14
EarlyStopping counter: 8 out of 50
[Epoch 11/500] train_loss: 0.06, valid_loss: 0.14
EarlyStopping counter: 9 out of 50
[Epoch 12/5

In [5]:
file = 'Model/eol_model_new_2.pth'
torch.save(model, file)
print(file)

Model/eol_model_new_2.pth


# Old

In [14]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_2')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device) # weights initialize by xavier normal distribution

model, train_loss, valid_loss = train_model(model_eol, writer, patience=30, lr=1e-2)

[Epoch 1/500] train_loss: 166.98, valid_loss: 386.55
Validation loss decreased (inf --> 386.550436).  Saving model ...
[Epoch 2/500] train_loss: 126.17, valid_loss: 223.17
Validation loss decreased (386.550436 --> 223.166453).  Saving model ...
[Epoch 3/500] train_loss: 118.79, valid_loss: 242.77
EarlyStopping counter: 1 out of 30
[Epoch 4/500] train_loss: 113.31, valid_loss: 256.18
EarlyStopping counter: 2 out of 30
[Epoch 5/500] train_loss: 110.67, valid_loss: 243.84
EarlyStopping counter: 3 out of 30
[Epoch 6/500] train_loss: 106.09, valid_loss: 251.87
EarlyStopping counter: 4 out of 30
[Epoch 7/500] train_loss: 100.15, valid_loss: 253.16
EarlyStopping counter: 5 out of 30
[Epoch 8/500] train_loss: 93.11, valid_loss: 224.08
EarlyStopping counter: 6 out of 30
[Epoch 9/500] train_loss: 88.13, valid_loss: 238.07
EarlyStopping counter: 7 out of 30
[Epoch 10/500] train_loss: 81.63, valid_loss: 244.68
EarlyStopping counter: 8 out of 30
[Epoch 11/500] train_loss: 76.04, valid_loss: 264.44


In [15]:
file = 'Model/eol_model6.pth'
torch.save(model, file)
print(file)

Model/eol_model6.pth


In [8]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_5')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=50, lr=1e-3, min_lr=1e-5)

[Epoch 1/500] train_loss: 330.10, valid_loss: 196.56
Validation loss decreased (inf --> 196.556061).  Saving model ...
[Epoch 2/500] train_loss: 135.02, valid_loss: 269.77
EarlyStopping counter: 1 out of 50
[Epoch 3/500] train_loss: 129.66, valid_loss: 233.09
EarlyStopping counter: 2 out of 50
[Epoch 4/500] train_loss: 124.09, valid_loss: 223.72
EarlyStopping counter: 3 out of 50
[Epoch 5/500] train_loss: 117.71, valid_loss: 239.03
EarlyStopping counter: 4 out of 50
[Epoch 6/500] train_loss: 112.30, valid_loss: 216.59
EarlyStopping counter: 5 out of 50
[Epoch 7/500] train_loss: 109.31, valid_loss: 230.81
EarlyStopping counter: 6 out of 50
[Epoch 8/500] train_loss: 107.18, valid_loss: 234.68
EarlyStopping counter: 7 out of 50
[Epoch 9/500] train_loss: 104.82, valid_loss: 214.95
EarlyStopping counter: 8 out of 50
[Epoch 10/500] train_loss: 101.60, valid_loss: 222.05
EarlyStopping counter: 9 out of 50
[Epoch 11/500] train_loss: 98.48, valid_loss: 229.63
EarlyStopping counter: 10 out of 50

In [9]:
file = 'Model/eol_model7.pth'
torch.save(model, file)
print(file)

Model/eol_model7.pth


In [4]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_6')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=50, lr=1e-2, cosine_period=10, min_lr=1e-5)

[Epoch 1/500] train_loss: 167.09, valid_loss: 250.55
Validation loss decreased (inf --> 250.546884).  Saving model ...
[Epoch 2/500] train_loss: 128.96, valid_loss: 233.59
Validation loss decreased (250.546884 --> 233.586453).  Saving model ...
[Epoch 3/500] train_loss: 118.45, valid_loss: 240.88
EarlyStopping counter: 1 out of 50
[Epoch 4/500] train_loss: 110.67, valid_loss: 239.36
EarlyStopping counter: 2 out of 50
[Epoch 5/500] train_loss: 101.52, valid_loss: 248.08
EarlyStopping counter: 3 out of 50
[Epoch 6/500] train_loss: 91.34, valid_loss: 225.31
Validation loss decreased (233.586453 --> 225.314473).  Saving model ...
[Epoch 7/500] train_loss: 79.67, valid_loss: 241.39
EarlyStopping counter: 1 out of 50
[Epoch 8/500] train_loss: 71.79, valid_loss: 229.20
EarlyStopping counter: 2 out of 50
[Epoch 9/500] train_loss: 66.00, valid_loss: 207.57
Validation loss decreased (225.314473 --> 207.565988).  Saving model ...
[Epoch 10/500] train_loss: 62.08, valid_loss: 212.19
EarlyStopping 

In [5]:
file = 'Model/eol_model8.pth'
torch.save(model, file)
print(file)

Model/eol_model8.pth


In [6]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_7')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=50, lr=1e-2, cosine_period=10, min_lr=1e-5, weight_decay=1e-2)

[Epoch 1/500] train_loss: 173.74, valid_loss: 328.01
Validation loss decreased (inf --> 328.005871).  Saving model ...
[Epoch 2/500] train_loss: 132.34, valid_loss: 270.94
Validation loss decreased (328.005871 --> 270.937398).  Saving model ...
[Epoch 3/500] train_loss: 124.40, valid_loss: 261.37
Validation loss decreased (270.937398 --> 261.373851).  Saving model ...
[Epoch 4/500] train_loss: 119.53, valid_loss: 234.60
Validation loss decreased (261.373851 --> 234.599645).  Saving model ...
[Epoch 5/500] train_loss: 111.72, valid_loss: 239.33
EarlyStopping counter: 1 out of 50
[Epoch 6/500] train_loss: 99.75, valid_loss: 228.22
Validation loss decreased (234.599645 --> 228.223046).  Saving model ...
[Epoch 7/500] train_loss: 92.03, valid_loss: 222.65
Validation loss decreased (228.223046 --> 222.647872).  Saving model ...
[Epoch 8/500] train_loss: 86.69, valid_loss: 227.43
EarlyStopping counter: 1 out of 50
[Epoch 9/500] train_loss: 79.90, valid_loss: 222.46
Validation loss decreased 

In [7]:
file = 'Model/eol_model9.pth'
torch.save(model, file)
print(file)

Model/eol_model9.pth


In [8]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_7')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=100, lr=1e-2, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2)

[Epoch 1/500] train_loss: 163.72, valid_loss: 319.04
Validation loss decreased (inf --> 319.035838).  Saving model ...
[Epoch 2/500] train_loss: 121.28, valid_loss: 235.35
Validation loss decreased (319.035838 --> 235.346567).  Saving model ...
[Epoch 3/500] train_loss: 114.27, valid_loss: 385.29
EarlyStopping counter: 1 out of 100
[Epoch 4/500] train_loss: 111.80, valid_loss: 227.25
Validation loss decreased (235.346567 --> 227.249518).  Saving model ...
[Epoch 5/500] train_loss: 104.54, valid_loss: 207.92
Validation loss decreased (227.249518 --> 207.917857).  Saving model ...
[Epoch 6/500] train_loss: 97.01, valid_loss: 223.59
EarlyStopping counter: 1 out of 100
[Epoch 7/500] train_loss: 89.83, valid_loss: 295.59
EarlyStopping counter: 2 out of 100
[Epoch 8/500] train_loss: 80.78, valid_loss: 220.50
EarlyStopping counter: 3 out of 100
[Epoch 9/500] train_loss: 74.81, valid_loss: 233.71
EarlyStopping counter: 4 out of 100
[Epoch 10/500] train_loss: 70.83, valid_loss: 228.09
EarlyStop

In [9]:
file = 'Model/eol_model10.pth'
torch.save(model, file)
print(file)

Model/eol_model10.pth


In [5]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_9')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=100, lr=1e-2, cosine_period=20, min_lr=1e-5, weight_decay=5*1e-2)

[Epoch 1/500] train_loss: 168.74, valid_loss: 229.65
Validation loss decreased (inf --> 229.654596).  Saving model ...
[Epoch 2/500] train_loss: 128.80, valid_loss: 239.85
EarlyStopping counter: 1 out of 200
[Epoch 3/500] train_loss: 123.12, valid_loss: 411.91
EarlyStopping counter: 2 out of 200
[Epoch 4/500] train_loss: 118.40, valid_loss: 331.04
EarlyStopping counter: 3 out of 200
[Epoch 5/500] train_loss: 115.12, valid_loss: 383.47
EarlyStopping counter: 4 out of 200
[Epoch 6/500] train_loss: 112.41, valid_loss: 424.22
EarlyStopping counter: 5 out of 200
[Epoch 7/500] train_loss: 110.88, valid_loss: 349.86
EarlyStopping counter: 6 out of 200
[Epoch 8/500] train_loss: 104.90, valid_loss: 269.63
EarlyStopping counter: 7 out of 200
[Epoch 9/500] train_loss: 102.05, valid_loss: 245.54
EarlyStopping counter: 8 out of 200
[Epoch 10/500] train_loss: 97.09, valid_loss: 251.19
EarlyStopping counter: 9 out of 200
[Epoch 11/500] train_loss: 92.65, valid_loss: 235.32
EarlyStopping counter: 10 o

In [6]:
file = 'Model/eol_model11.pth'
torch.save(model, file)
print(file)

Model/eol_model11.pth


In [7]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_10')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=100, lr=1e-2, cosine_period=20, min_lr=1e-4, weight_decay=3*1e-2)

[Epoch 1/500] train_loss: 171.02, valid_loss: 280.12
Validation loss decreased (inf --> 280.116537).  Saving model ...
[Epoch 2/500] train_loss: 126.35, valid_loss: 297.77
EarlyStopping counter: 1 out of 100
[Epoch 3/500] train_loss: 121.36, valid_loss: 337.63
EarlyStopping counter: 2 out of 100
[Epoch 4/500] train_loss: 115.63, valid_loss: 230.17
Validation loss decreased (280.116537 --> 230.166738).  Saving model ...
[Epoch 5/500] train_loss: 110.70, valid_loss: 273.03
EarlyStopping counter: 1 out of 100
[Epoch 6/500] train_loss: 103.97, valid_loss: 225.84
Validation loss decreased (230.166738 --> 225.844136).  Saving model ...
[Epoch 7/500] train_loss: 99.29, valid_loss: 249.57
EarlyStopping counter: 1 out of 100
[Epoch 8/500] train_loss: 90.37, valid_loss: 240.91
EarlyStopping counter: 2 out of 100
[Epoch 9/500] train_loss: 85.51, valid_loss: 280.22
EarlyStopping counter: 3 out of 100
[Epoch 10/500] train_loss: 82.62, valid_loss: 255.21
EarlyStopping counter: 4 out of 100
[Epoch 11

In [8]:
file = 'Model/eol_model12.pth'
torch.save(model, file)
print(file)

Model/eol_model12.pth


In [9]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_11')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=100, lr=1e-3, cosine_period=20, min_lr=1e-5, weight_decay=5*1e-2)

[Epoch 1/500] train_loss: 351.30, valid_loss: 185.27
Validation loss decreased (inf --> 185.267920).  Saving model ...
[Epoch 2/500] train_loss: 134.88, valid_loss: 266.87
EarlyStopping counter: 1 out of 100
[Epoch 3/500] train_loss: 123.45, valid_loss: 303.06
EarlyStopping counter: 2 out of 100
[Epoch 4/500] train_loss: 118.74, valid_loss: 225.24
EarlyStopping counter: 3 out of 100
[Epoch 5/500] train_loss: 109.06, valid_loss: 239.95
EarlyStopping counter: 4 out of 100
[Epoch 6/500] train_loss: 101.86, valid_loss: 261.69
EarlyStopping counter: 5 out of 100
[Epoch 7/500] train_loss: 95.33, valid_loss: 251.36
EarlyStopping counter: 6 out of 100
[Epoch 8/500] train_loss: 86.40, valid_loss: 211.69
EarlyStopping counter: 7 out of 100
[Epoch 9/500] train_loss: 79.18, valid_loss: 227.74
EarlyStopping counter: 8 out of 100
[Epoch 10/500] train_loss: 74.88, valid_loss: 225.58
EarlyStopping counter: 9 out of 100
[Epoch 11/500] train_loss: 68.42, valid_loss: 239.28
EarlyStopping counter: 10 out 

In [10]:
file = 'Model/eol_model13.pth'
torch.save(model, file)
print(file)

Model/eol_model13.pth


In [11]:
batch_size=64
eol_train_loader = DataLoader(eol_train_dataset, batch_size=batch_size, shuffle=True)
eol_test_loader = DataLoader(eol_test_dataset, batch_size=batch_size, shuffle=False)
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_12')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=100, lr=1e-2, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2)

[Epoch 1/500] train_loss: 200.70, valid_loss: 278.15
Validation loss decreased (inf --> 278.154435).  Saving model ...
[Epoch 2/500] train_loss: 123.84, valid_loss: 269.72
Validation loss decreased (278.154435 --> 269.721180).  Saving model ...
[Epoch 3/500] train_loss: 120.85, valid_loss: 260.15
Validation loss decreased (269.721180 --> 260.146641).  Saving model ...
[Epoch 4/500] train_loss: 115.46, valid_loss: 232.23
Validation loss decreased (260.146641 --> 232.228235).  Saving model ...
[Epoch 5/500] train_loss: 111.38, valid_loss: 399.50
EarlyStopping counter: 1 out of 100
[Epoch 6/500] train_loss: 101.92, valid_loss: 288.85
EarlyStopping counter: 2 out of 100
[Epoch 7/500] train_loss: 96.96, valid_loss: 256.77
EarlyStopping counter: 3 out of 100
[Epoch 8/500] train_loss: 84.11, valid_loss: 256.57
EarlyStopping counter: 4 out of 100
[Epoch 9/500] train_loss: 78.02, valid_loss: 243.96
EarlyStopping counter: 5 out of 100
[Epoch 10/500] train_loss: 73.61, valid_loss: 258.74
EarlySto

In [13]:
batch_size=32
eol_train_loader = DataLoader(eol_train_dataset, batch_size=batch_size, shuffle=True)
eol_test_loader = DataLoader(eol_test_dataset, batch_size=batch_size, shuffle=False)
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_14')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=30, lr=5*1e-2, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.5)

[Epoch 1/500] train_loss: 79.80, valid_loss: 147.53
Validation loss decreased (inf --> 147.525052).  Saving model ...
[Epoch 2/500] train_loss: 68.13, valid_loss: 131.05
Validation loss decreased (147.525052 --> 131.051348).  Saving model ...
[Epoch 3/500] train_loss: 64.27, valid_loss: 181.98
EarlyStopping counter: 1 out of 30
[Epoch 4/500] train_loss: 63.91, valid_loss: 164.78
EarlyStopping counter: 2 out of 30
[Epoch 5/500] train_loss: 62.01, valid_loss: 133.32
EarlyStopping counter: 3 out of 30
[Epoch 6/500] train_loss: 61.41, valid_loss: 147.65
EarlyStopping counter: 4 out of 30
[Epoch 7/500] train_loss: 58.42, valid_loss: 140.79
EarlyStopping counter: 5 out of 30
[Epoch 8/500] train_loss: 55.40, valid_loss: 130.77
Validation loss decreased (131.051348 --> 130.768727).  Saving model ...
[Epoch 9/500] train_loss: 53.03, valid_loss: 119.08
Validation loss decreased (130.768727 --> 119.083753).  Saving model ...
[Epoch 10/500] train_loss: 49.41, valid_loss: 125.35
EarlyStopping count

In [14]:
file = 'Model/eol_model14.pth'
torch.save(model, file)
print(file)

Model/eol_model14.pth


In [15]:
batch_size=32
eol_train_loader = DataLoader(eol_train_dataset, batch_size=batch_size, shuffle=True)
eol_test_loader = DataLoader(eol_test_dataset, batch_size=batch_size, shuffle=False)
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_15')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=30, lr=5*1e-3, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.5)

[Epoch 1/500] train_loss: 96.48, valid_loss: 131.81
Validation loss decreased (inf --> 131.809647).  Saving model ...
[Epoch 2/500] train_loss: 65.09, valid_loss: 174.94
EarlyStopping counter: 1 out of 30
[Epoch 3/500] train_loss: 61.45, valid_loss: 145.95
EarlyStopping counter: 2 out of 30
[Epoch 4/500] train_loss: 60.03, valid_loss: 125.47
Validation loss decreased (131.809647 --> 125.471477).  Saving model ...
[Epoch 5/500] train_loss: 56.89, valid_loss: 141.24
EarlyStopping counter: 1 out of 30
[Epoch 6/500] train_loss: 53.88, valid_loss: 137.97
EarlyStopping counter: 2 out of 30
[Epoch 7/500] train_loss: 49.44, valid_loss: 129.33
EarlyStopping counter: 3 out of 30
[Epoch 8/500] train_loss: 45.75, valid_loss: 119.74
Validation loss decreased (125.471477 --> 119.741732).  Saving model ...
[Epoch 9/500] train_loss: 42.80, valid_loss: 115.48
Validation loss decreased (119.741732 --> 115.476114).  Saving model ...
[Epoch 10/500] train_loss: 40.36, valid_loss: 118.73
EarlyStopping count

In [16]:
file = 'Model/eol_model15.pth'
torch.save(model, file)
print(file)

Model/eol_model15.pth


In [17]:
batch_size=32
eol_train_loader = DataLoader(eol_train_dataset, batch_size=batch_size, shuffle=True)
eol_test_loader = DataLoader(eol_test_dataset, batch_size=batch_size, shuffle=False)
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_16')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=30, lr=1e-3, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.5)

[Epoch 1/500] train_loss: 167.33, valid_loss: 120.47
Validation loss decreased (inf --> 120.473916).  Saving model ...
[Epoch 2/500] train_loss: 65.51, valid_loss: 90.68
Validation loss decreased (120.473916 --> 90.677842).  Saving model ...
[Epoch 3/500] train_loss: 60.14, valid_loss: 135.94
EarlyStopping counter: 1 out of 30
[Epoch 4/500] train_loss: 54.19, valid_loss: 106.27
EarlyStopping counter: 2 out of 30
[Epoch 5/500] train_loss: 47.19, valid_loss: 113.36
EarlyStopping counter: 3 out of 30
[Epoch 6/500] train_loss: 41.34, valid_loss: 106.44
EarlyStopping counter: 4 out of 30
[Epoch 7/500] train_loss: 35.71, valid_loss: 111.62
EarlyStopping counter: 5 out of 30
[Epoch 8/500] train_loss: 33.40, valid_loss: 117.98
EarlyStopping counter: 6 out of 30
[Epoch 9/500] train_loss: 30.95, valid_loss: 88.83
Validation loss decreased (90.677842 --> 88.833271).  Saving model ...
[Epoch 10/500] train_loss: 29.17, valid_loss: 104.47
EarlyStopping counter: 1 out of 30
[Epoch 11/500] train_loss:

In [18]:
file = 'Model/eol_model16.pth'
torch.save(model, file)
print(file)

Model/eol_model16.pth


In [4]:
batch_size=32
eol_train_loader = DataLoader(eol_train_dataset, batch_size=batch_size, shuffle=True)
eol_test_loader = DataLoader(eol_test_dataset, batch_size=batch_size, shuffle=False)
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_18')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=30, lr=1e-3, cosine_period=10, min_lr=1e-5, weight_decay=3*1e-2, delta_huber=0.5)

[Epoch 1/500] train_loss: 169.92, valid_loss: 144.14
Validation loss decreased (inf --> 144.144697).  Saving model ...
[Epoch 2/500] train_loss: 63.55, valid_loss: 105.68
Validation loss decreased (144.144697 --> 105.684525).  Saving model ...
[Epoch 3/500] train_loss: 54.06, valid_loss: 132.76
EarlyStopping counter: 1 out of 30
[Epoch 4/500] train_loss: 45.99, valid_loss: 106.18
EarlyStopping counter: 2 out of 30
[Epoch 5/500] train_loss: 40.03, valid_loss: 111.14
EarlyStopping counter: 3 out of 30
[Epoch 6/500] train_loss: 34.93, valid_loss: 119.01
EarlyStopping counter: 4 out of 30
[Epoch 7/500] train_loss: 32.21, valid_loss: 119.74
EarlyStopping counter: 5 out of 30
[Epoch 8/500] train_loss: 28.23, valid_loss: 103.40
Validation loss decreased (105.684525 --> 103.402545).  Saving model ...
[Epoch 9/500] train_loss: 26.25, valid_loss: 105.88
EarlyStopping counter: 1 out of 30
[Epoch 10/500] train_loss: 25.78, valid_loss: 107.23
EarlyStopping counter: 2 out of 30
[Epoch 11/500] train_

In [5]:
batch_size=32
eol_train_loader = DataLoader(eol_train_dataset, batch_size=batch_size, shuffle=True)
eol_test_loader = DataLoader(eol_test_dataset, batch_size=batch_size, shuffle=False)
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_19')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=30, lr=1e-3, cosine_period=10, min_lr=1e-5, weight_decay=1e-1, delta_huber=0.5)

[Epoch 1/500] train_loss: 176.91, valid_loss: 109.64
Validation loss decreased (inf --> 109.644958).  Saving model ...
[Epoch 2/500] train_loss: 63.06, valid_loss: 105.07
Validation loss decreased (109.644958 --> 105.073775).  Saving model ...
[Epoch 3/500] train_loss: 52.89, valid_loss: 104.97
Validation loss decreased (105.073775 --> 104.973053).  Saving model ...
[Epoch 4/500] train_loss: 44.52, valid_loss: 109.68
EarlyStopping counter: 1 out of 30
[Epoch 5/500] train_loss: 40.08, valid_loss: 113.43
EarlyStopping counter: 2 out of 30
[Epoch 6/500] train_loss: 35.93, valid_loss: 127.99
EarlyStopping counter: 3 out of 30
[Epoch 7/500] train_loss: 33.11, valid_loss: 119.93
EarlyStopping counter: 4 out of 30
[Epoch 8/500] train_loss: 30.59, valid_loss: 97.84
Validation loss decreased (104.973053 --> 97.838309).  Saving model ...
[Epoch 9/500] train_loss: 28.59, valid_loss: 100.45
EarlyStopping counter: 1 out of 30
[Epoch 10/500] train_loss: 26.75, valid_loss: 91.30
Validation loss decre

In [6]:
batch_size=32
eol_train_loader = DataLoader(eol_train_dataset, batch_size=batch_size, shuffle=True)
eol_test_loader = DataLoader(eol_test_dataset, batch_size=batch_size, shuffle=False)
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_20')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=30, lr=1e-3, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.3)

[Epoch 1/500] train_loss: 102.94, valid_loss: 60.09
Validation loss decreased (inf --> 60.088982).  Saving model ...
[Epoch 2/500] train_loss: 39.03, valid_loss: 58.36
Validation loss decreased (60.088982 --> 58.360016).  Saving model ...
[Epoch 3/500] train_loss: 34.79, valid_loss: 58.37
EarlyStopping counter: 1 out of 30
[Epoch 4/500] train_loss: 29.78, valid_loss: 54.84
Validation loss decreased (58.360016 --> 54.841113).  Saving model ...
[Epoch 5/500] train_loss: 25.58, valid_loss: 62.51
EarlyStopping counter: 1 out of 30
[Epoch 6/500] train_loss: 22.46, valid_loss: 64.88
EarlyStopping counter: 2 out of 30
[Epoch 7/500] train_loss: 20.53, valid_loss: 64.93
EarlyStopping counter: 3 out of 30
[Epoch 8/500] train_loss: 18.95, valid_loss: 56.01
EarlyStopping counter: 4 out of 30
[Epoch 9/500] train_loss: 18.10, valid_loss: 58.81
EarlyStopping counter: 5 out of 30
[Epoch 10/500] train_loss: 17.00, valid_loss: 57.53
EarlyStopping counter: 6 out of 30
[Epoch 11/500] train_loss: 16.87, va

In [7]:
file = 'Model/eol_model17.pth'
torch.save(model, file)
print(file)

Model/eol_model17.pth


In [8]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_21')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=50, lr=1e-2, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.3)

[Epoch 1/500] train_loss: 51.00, valid_loss: 110.94
Validation loss decreased (inf --> 110.940158).  Saving model ...
[Epoch 2/500] train_loss: 38.26, valid_loss: 76.51
Validation loss decreased (110.940158 --> 76.510541).  Saving model ...
[Epoch 3/500] train_loss: 36.41, valid_loss: 82.92
EarlyStopping counter: 1 out of 50
[Epoch 4/500] train_loss: 34.42, valid_loss: 70.99
Validation loss decreased (76.510541 --> 70.989640).  Saving model ...
[Epoch 5/500] train_loss: 33.66, valid_loss: 80.15
EarlyStopping counter: 1 out of 50
[Epoch 6/500] train_loss: 30.51, valid_loss: 85.80
EarlyStopping counter: 2 out of 50
[Epoch 7/500] train_loss: 28.15, valid_loss: 74.37
EarlyStopping counter: 3 out of 50
[Epoch 8/500] train_loss: 25.49, valid_loss: 76.40
EarlyStopping counter: 4 out of 50
[Epoch 9/500] train_loss: 23.04, valid_loss: 72.57
EarlyStopping counter: 5 out of 50
[Epoch 10/500] train_loss: 21.74, valid_loss: 71.24
EarlyStopping counter: 6 out of 50
[Epoch 11/500] train_loss: 21.65, 

In [9]:
file = 'Model/eol_model18.pth'
torch.save(model, file)
print(file)

Model/eol_model18.pth


In [11]:
from SeversonDataset_preprocess import dataset_preprocessing_DimReduction
dataset_preprocessing_DimReduction(train_val_split=0.8, seed=100, folder='Dataset/')

In [14]:
batch_size=32
# load dataset
eol_train_dataset = DimReduction_dataset(train=True, pred_target='EOL', norm=False)
eol_test_dataset = DimReduction_dataset(train=False, pred_target='EOL', norm=False)
eol_train_loader = DataLoader(eol_train_dataset, batch_size=batch_size, shuffle=True)
eol_test_loader = DataLoader(eol_test_dataset, batch_size=batch_size, shuffle=False)

# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_23')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=50, lr=1e-2, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.3)

[Epoch 1/500] train_loss: 53.68, valid_loss: 116.87
Validation loss decreased (inf --> 116.874578).  Saving model ...
[Epoch 2/500] train_loss: 40.58, valid_loss: 57.07
Validation loss decreased (116.874578 --> 57.068756).  Saving model ...
[Epoch 3/500] train_loss: 36.85, valid_loss: 83.08
EarlyStopping counter: 1 out of 50
[Epoch 4/500] train_loss: 35.69, valid_loss: 56.09
Validation loss decreased (57.068756 --> 56.090119).  Saving model ...
[Epoch 5/500] train_loss: 33.72, valid_loss: 56.90
EarlyStopping counter: 1 out of 50
[Epoch 6/500] train_loss: 31.72, valid_loss: 61.14
EarlyStopping counter: 2 out of 50
[Epoch 7/500] train_loss: 29.44, valid_loss: 59.65
EarlyStopping counter: 3 out of 50
[Epoch 8/500] train_loss: 27.01, valid_loss: 57.75
EarlyStopping counter: 4 out of 50
[Epoch 9/500] train_loss: 24.42, valid_loss: 56.57
EarlyStopping counter: 5 out of 50
[Epoch 10/500] train_loss: 21.96, valid_loss: 52.73
Validation loss decreased (56.090119 --> 52.734074).  Saving model ..

In [15]:
file = 'Model/eol_model19.pth'
torch.save(model, file)
print(file)

Model/eol_model19.pth


In [16]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_24')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=50, lr=1e-2, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.3)

[Epoch 1/500] train_loss: 54.36, valid_loss: 72.25
Validation loss decreased (inf --> 72.245442).  Saving model ...
[Epoch 2/500] train_loss: 41.74, valid_loss: 42.96
Validation loss decreased (72.245442 --> 42.956809).  Saving model ...
[Epoch 3/500] train_loss: 39.17, valid_loss: 64.47
EarlyStopping counter: 1 out of 50
[Epoch 4/500] train_loss: 37.42, valid_loss: 41.70
Validation loss decreased (42.956809 --> 41.698161).  Saving model ...
[Epoch 5/500] train_loss: 35.66, valid_loss: 66.38
EarlyStopping counter: 1 out of 50
[Epoch 6/500] train_loss: 33.61, valid_loss: 57.78
EarlyStopping counter: 2 out of 50
[Epoch 7/500] train_loss: 30.53, valid_loss: 63.39
EarlyStopping counter: 3 out of 50
[Epoch 8/500] train_loss: 27.74, valid_loss: 53.69
EarlyStopping counter: 4 out of 50
[Epoch 9/500] train_loss: 24.52, valid_loss: 52.59
EarlyStopping counter: 5 out of 50
[Epoch 10/500] train_loss: 22.21, valid_loss: 54.10
EarlyStopping counter: 6 out of 50
[Epoch 11/500] train_loss: 21.50, val

In [17]:
file = 'Model/eol_model20.pth'
torch.save(model, file)
print(file)

Model/eol_model20.pth


In [18]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_25')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=50, lr=1e-2, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.3)

[Epoch 1/500] train_loss: 53.99, valid_loss: 45.28
Validation loss decreased (inf --> 45.284342).  Saving model ...
[Epoch 2/500] train_loss: 44.04, valid_loss: 76.05
EarlyStopping counter: 1 out of 50
[Epoch 3/500] train_loss: 40.03, valid_loss: 52.98
EarlyStopping counter: 2 out of 50
[Epoch 4/500] train_loss: 38.67, valid_loss: 41.36
Validation loss decreased (45.284342 --> 41.364028).  Saving model ...
[Epoch 5/500] train_loss: 37.74, valid_loss: 42.50
EarlyStopping counter: 1 out of 50
[Epoch 6/500] train_loss: 35.58, valid_loss: 75.04
EarlyStopping counter: 2 out of 50
[Epoch 7/500] train_loss: 33.10, valid_loss: 56.92
EarlyStopping counter: 3 out of 50
[Epoch 8/500] train_loss: 30.08, valid_loss: 55.70
EarlyStopping counter: 4 out of 50
[Epoch 9/500] train_loss: 27.51, valid_loss: 50.83
EarlyStopping counter: 5 out of 50
[Epoch 10/500] train_loss: 25.33, valid_loss: 50.14
EarlyStopping counter: 6 out of 50
[Epoch 11/500] train_loss: 24.81, valid_loss: 51.38
EarlyStopping counter

In [19]:
file = 'Model/eol_model21.pth'
torch.save(model, file)
print(file)

Model/eol_model21.pth


In [ ]:
# TensorBoard
writer = SummaryWriter('runs/eol_earlystop_25')
# load model
model_eol = DimReduction_1(in_ch=4, out_ch=1).to(device)

model, train_loss, valid_loss = train_model(model_eol, writer, patience=50, lr=1e-2, cosine_period=10, min_lr=1e-5, weight_decay=5*1e-2, delta_huber=0.3)